First we are going to parse the annotation file and see what we can learn...

In [1]:
from rdflib import Graph, URIRef
from rdflib.namespace import DCTERMS

# use this URI to identify delayed variables - not the perfect URI, but will do for now
#     This is actually "delayed differential equation model" from the MAMO ontology
delay_variable_uri = URIRef('http://identifiers.org/mamo/MAMO_0000089')
# a "readout" variable that we maybe want to connect to something external?
timecourse_readout_uri = URIRef('http://identifiers.org/mamo/MAMO_0000031')

annotation_file = 'models/sine_approximations-updated-ids--annotations.ttl'
g = Graph().parse(annotation_file)

# find all delayed variables
delayed_variables = []
for d in g.subjects(DCTERMS.type, delay_variable_uri):
    delayed_variables.append(str(d))
    
print(delayed_variables)

# find all timecourse readouts
readout_variables = []
for d in g.subjects(DCTERMS.type, timecourse_readout_uri):
    readout_variables.append(str(d))
    
print(readout_variables)

['file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da77']
['file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da7c']


We're going to use the "model" file from the first delay variable and only continue if all annotations use the same model...

In [2]:
from urllib.parse import urlparse

model_uri = delayed_variables[0]
model_url = urlparse(model_uri)
model_file = model_url.path

delayed_ids = []
for v in delayed_variables:
    url = urlparse(v)
    if url.path != model_file:
        print("found an unexpected model file for delayed variable?!")
        exit
    delayed_ids.append(url.fragment)
    
readout_ids = []
for v in readout_variables:
    url = urlparse(v)
    if url.path != model_file:
        print("found an unexpected model file for readout variable?!")
        exit
    readout_ids.append(url.fragment)

Now we have the model file and the IDs for the variables in that model that we want to do stuff with. So we can parse the model and see if we can find the variables.

In [3]:
import cellml

# on windows getting a leading '/' in the filename which libCellML doesn't like...
fixed_model_file = model_file[1:]

# parse the model in non-strict mode to allow non CellML 2.0 models
model = cellml.parse_model(fixed_model_file, False)

# and make an annotator for this model
from libcellml import Annotator
annotator = Annotator()
annotator.setModel(model)

# map our IDs to the actual variables
annotated_variables = []
for i in delayed_ids:
    # get the variable (will fail if id doesn't correspond to a variable in the model)
    v = annotator.variable(i)
    if v == None:
        print('Unable to find a delay variable with the id {} in the given model...'.format(i))
        exit
    annotated_variables.append([v, delay_variable_uri])
    
for i in readout_ids:
    # get the variable (will fail if id doesn't correspond to a variable in the model)
    v = annotator.variable(i)
    if v == None:
        print('Unable to find a readout variable with the id {} in the given model...'.format(i))
        exit
    annotated_variables.append([v, timecourse_readout_uri])

# TODO:
Need to work out how to map the annotations through to the variables in the generated code....

Generate C code for the model.

In [5]:
import os 
model_dir = os.path.dirname(fixed_model_file)

# resolve imports, in non-strict mode
importer = cellml.resolve_imports(model, model_dir, False)
# need a flattened model for analysing
flat_model = cellml.flatten_model(model, importer)

from libcellml import Analyser, AnalyserModel, Generator, GeneratorProfile

# analyse the model
a = Analyser()
a.analyseModel(flat_model)
analysed_model = a.model()
print(analysed_model.type())

g = Generator()
profile = GeneratorProfile(GeneratorProfile.Profile.C)
g.setProfile(profile)
g.setModel(analysed_model)

# print the C header code
print('header code:')
print(g.interfaceCode())

# print the C implementation code
print('implementation code:')
print(g.implementationCode())

no unresolved imports.
2
header code:
/* The content of this file was generated using the C profile of libCellML 0.3.104. */

#pragma once

#include <stddef.h>

extern const char VERSION[];
extern const char LIBCELLML_VERSION[];

extern const size_t STATE_COUNT;
extern const size_t VARIABLE_COUNT;

typedef enum {
    VARIABLE_OF_INTEGRATION,
    STATE,
    CONSTANT,
    COMPUTED_CONSTANT,
    ALGEBRAIC
} VariableType;

typedef struct {
    char name[27];
    char units[14];
    char component[21];
    VariableType type;
} VariableInfo;

extern const VariableInfo VOI_INFO;
extern const VariableInfo STATE_INFO[];
extern const VariableInfo VARIABLE_INFO[];

double * createStatesArray();
double * createVariablesArray();
void deleteArray(double *array);

void initialiseVariables(double *states, double *variables);
void computeComputedConstants(double *variables);
void computeRates(double voi, double *states, double *rates, double *variables);
void computeVariables(double voi, double *states